In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import pandas as pd
from cartopy.io import shapereader
import pandas as pd
import matplotlib.patches as patches
import paraguayfloodspy.visualize as viz

In [ ]:
%matplotlib inline

Load in relevant parameters from the `config` directory, then print them to ensure we've gotten them right

In [ ]:
%run ../config/RioParaguay.mk
%run ../config/WeatherTypes.mk
print(RPEAST, RPWEST, RPSOUTH, RPNORTH)
print(WTEAST, WTWEST, WTSOUTH, WTNORTH)

Save figures?

In [ ]:
savefigs = True

Rivers to plot

In [ ]:
rivers = shapereader.natural_earth(
    category='physical',
    name='rivers_lake_centerlines',
    resolution='10m'
)
rivers = shapereader.Reader(rivers)

Get the streamflow gauge station descriptions

In [ ]:
desc = pd.read_csv('../data/given/station_description.csv')

Define a couple of boxes that we'll show on the map

In [ ]:
def wt_region():
    rec = patches.Rectangle(
        (WTWEST-360, WTSOUTH),
        WTEAST - WTWEST,
        WTNORTH - WTSOUTH,
        color = 'red', label = 'Weather Typing Region',
        fill = None, linewidth=2
    )
    return(rec)

In [ ]:
def rpy_rgn():
    rec = patches.Rectangle(
        (RPWEST-360, RPSOUTH),
        RPEAST - RPWEST,
        RPNORTH - RPSOUTH,
        color = 'black', label = 'Lower PY River Basin',
        fill = None, linewidth=2
        )
    return(rec)

Get elevation data

In [ ]:
elev = xr.open_dataarray('../data/accessed/elevation.nc').sel(X=slice(-85, -32.5), Y=slice(-50,15))
Xe, Ye = np.meshgrid(elev.X, elev.Y)
log_elev = np.log10(elev)
log_elev = np.ma.masked_invalid(log_elev)
elev = np.ma.masked_invalid(elev)

Code to create the axes

In [ ]:
fig, axes = viz.SetupAxes(ncol = 2, nax = 1, proj = ccrs.PlateCarree(), figsize = [10,7.5])
# LEFT PLOT: Show Study area in context of South America
ax = axes[0]
C = ax.pcolormesh(Xe, Ye, log_elev, vmin=-0.5, vmax=4, cmap='terrain')
ax.add_patch(wt_region())
ax.add_patch(rpy_rgn())
ax.legend(loc='lower right')

# RIGHT PLOT: Lower Paraguay River
ax = axes[1]
C = ax.pcolormesh(Xe, Ye, log_elev, vmin=-0.5, vmax=4, cmap='terrain')
ax.add_patch(rpy_rgn())
rivernum_plot = [36, 1032, 1125, 294]
for rec in rivers.records():
    name = rec.attributes['name']
    num = rec.attributes['rivernum']
    if np.in1d(num, rivernum_plot):
        ax.add_geometries( [rec.geometry], ccrs.PlateCarree(), edgecolor='blue', facecolor='none')
    pass
for i in range(desc.shape[0]):
    ax.scatter(x=desc['lon'][i], y = desc['lat'][i], color='k')
    ax.text(x=desc['lon'][i] + 0.25, y = desc['lat'][i] - 0.1, s = desc['short_name'][i], color='black')

ax.legend(loc = 'upper left')

# Countries, extents, ticks
viz.FormatAxes(axes[0], coast=True, grid=True, border=True, river = False, feature_list = [], states=False,
           extent = [-85, -32.5, -50, 15], ticks=[np.linspace(-180, 180, 37), np.linspace(-90, 90, 19)])
viz.FormatAxes(axes[1], coast=True, grid=True, border=True, river = False, feature_list = [], states=False,
           extent = [-67.5, -52.5, -30, -12.5], ticks=[np.linspace(-180, 180, 73), np.linspace(-90, 90, 37)])
plt.tight_layout()
fig.subplots_adjust(right=0.9)
cax1 = fig.add_axes([0.92, 0.2, 0.02, 0.6])
cb = plt.colorbar(C, cax=cax1)
cb.set_label(r"$\log_{10}$ Elevation [m]", rotation=270)
cb.ax.get_yaxis().labelpad = 20

if savefigs:
    fig.savefig("../figs/StudyArea.jpg", bbox_inches='tight', dpi=500)